In [25]:
from dotenv import load_dotenv
import os
from crewai import Agent, Task, Crew
from langchain_community.llms import Ollama
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
import warnings

# Load environment variables from .env file
load_dotenv()

# Warning control
warnings.filterwarnings('ignore')

llm = Ollama(
    model = "llama3",
    base_url = "http://localhost:11434"
)

support_agent = Agent(
    role="Senior Support Representative",
    goal="Be the most friendly and helpful support representative in your team",
    backstory=(
        "You work at crewAI (https://crewai.com) and are now working on providing "
        "support to {customer}, a super important customer for your company. "
        "You need to make sure that you provide the best support! "
        "Make sure to provide full complete answers, and make no assumptions."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

support_quality_assurance_agent = Agent(
    role="Support Quality Assurance Specialist",
    goal="Get recognition for providing the best support quality assurance in your team",
    backstory=(
        "You work at crewAI (https://crewai.com) and are now working with your team "
        "on a request from {customer} ensuring that the support representative is "
        "providing the best support possible.\n"
        "You need to make sure that the support representative is providing full "
        "complete answers, and make no assumptions."
    ),
    verbose=True,
    llm=llm
)

serper_tool = SerperDevTool()

docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

website_search_tool = WebsiteSearchTool(
    config=dict(
        llm=dict(
            provider="ollama",
            config=dict(
                model="llama3",
                base_url="http://localhost:11434",
            ),
        ),
        embedder=dict(
            provider="google",
            config=dict(
                model="models/embedding-001",
                task_type="retrieval_document",
            ),
        ),
    )
)

inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
        "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
        "Make sure to use everything you know. "
        "You must provide the best support possible. "
        "You must strive to provide a complete and accurate response to the customer's inquiry."
    ),
    expected_output=(
        "A detailed, informative response to the "
        "customer's inquiry that addresses all aspects of their question.\n"
        "The response should include references to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
        "leaving no questions unanswered and maintaining a helpful and friendly tone throughout."
    ),
    tools=[docs_scrape_tool, serper_tool, website_search_tool],
    agent=support_agent,
)

quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
        "high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry have been addressed "
        "thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to find the information, "
        "ensuring the response is well-supported and leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response ready to be sent to the customer.\n"
        "This response should fully address the customer's inquiry, incorporating all "
        "relevant feedback and improvements.\n"
        "Don't be too formal, we are a chill and cool company "
        "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2
)

inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}

result = crew.kickoff(inputs=inputs)
from IPython.display import Markdown
Markdown(result)

2024-07-11 13:22:06,372 - 65904 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know. You must provide the best support possible. You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
Thought: I understand that DeepLearningAI needs help setting up a Crew and kicking it off, specifically adding memory to their crew. I need to provide guidance on how to do this.

Action: Read website content
Action Input: {"url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"I apologize for the mistake! It seems I misunderstood the format. Let me try again.

Thought: 
Action: Read website content
Action Input: {"url": "https://docs

** To delegate work to a co-worker, I would like to assign the task "Review the drafted response and ensure it meets the high-quality standards expected for customer support" to the Senior Support Representative. The context for this task includes detailed steps on how to set up a Crew and add additional memory: 

"To set up a Crew and kick it off with additional memory, follow these steps:
1. Log in to your CrewAI account and navigate to the \"Crews\" tab.
2. Click on the desired Crew to access its settings page.
3. Scroll down to the \"Memory\" section and click the \"Edit\" button next to \"Total Memory.\" 
4. Enter the desired amount of memory (in GB) you'd like to add to your Crew.
5. Click the \"Save\" button to apply the changes.
That's it! Your Crew should now have the additional memory you specified. If you encounter any issues or have further questions, feel free to reach out to our support team for assistance."

Please let me know if this meets the expected criteria!